## StarTracker Narrow to Fast Camera offsets

Craig Lage - 05-Apr-23

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
from scipy.ndimage import median_filter
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import pandas as pd
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.geom import Point2D, SpherePoint, Angle, AngleUnit

import lsst.afw.image as afwImage
from lsst.geom import SpherePoint
from lsst.geom import Angle as afwAngle
from astroquery.simbad import Simbad

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

# First, calculate the offsets by reading the astrometry solutions from 2023-03-21

In [ ]:
# Read the RubinTV JSON file
filename = f'/scratch/cslage/starTracker/startracker_2023-03-21.json'
df = pd.read_json(filename)
df = df.transpose()
df = df.dropna()

# Calculate the AltAz offsets
deltaAlts = []
deltaAzCosAlts = []
for seqNum in df.index.values.tolist():
    row = df.loc[seqNum]
    deltaAz = (row['Calculated Az fast'] - row['Calculated Az']) * 3600.0
    # To get the actual offset on the sky, we need to multiply by cos(alt)
    deltaAzCosAlt = deltaAz * np.cos(row['Calculated Alt'] * np.pi / 180.0)
    deltaAlt = (row['Calculated Alt fast'] - row['Calculated Alt']) * 3600.0
    deltaAlts.append(deltaAlt)
    deltaAzCosAlts.append(deltaAzCosAlt)
    delta = np.sqrt(deltaAlt**2 + deltaAzCosAlt**2)
    #print(seqNum, deltaAlt, deltaAzCosAlt, delta)
median_offset = [np.median(deltaAlts), np.median(deltaAzCosAlts)]
# Now plot it
%matplotlib inline
plt.subplot(1,2,1)
plt.title("deltaAlts")
plt.hist(deltaAlts)
plt.text(-150,70,f"Median={median_offset[0]:.1f}")
plt.xlim(-600, 600)
plt.xlabel("Fast-Narrow deltaAlt(arcseconds)")
plt.subplot(1,2,2)
plt.title("deltaAzCosAlts")
plt.hist(deltaAzCosAlts)
plt.text(-150,70,f"Median={median_offset[1]:.1f}")
plt.xlim(-600, 600)
plt.xlabel("Fast-Narrow deltaAzCosAlt(arcseconds)")
plt.savefig(f'/home/craiglagegit/DATA/Fast_Camera_Offset_20230321.png')

## Now look at a representative image and plot the offsets as a check

### First solve the narrow image

In [ ]:
cameras = [['Narrow', 102]]
dates = [20230321]
seqNum = 1254
for [camera, num] in cameras:
    for date in dates:
        year = int(date/10000)
        month = int((date - 10000 * year)/100)
        day = int((date - 10000 * year - 100 * month))
        path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
        filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
        narrow_exp = starTrackerFileToExposure(filename)
        results = runCharactierizeImage(narrow_exp, 5, 25)
        filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
        # Need to loosen percentageScaleError because header SECPIX is wrong
        solverResults = solver.run(narrow_exp, filteredCat, percentageScaleError=60.0, isWideField=True)
narrow_wcs = solverResults.wcs

### Next solve the fast image

In [ ]:
# Code from Merlin Fisher-Levine to solve fast camera images
def runCharactierizeImage(exp, snr, minPix):
    """Run the image characterization task, finding only bright sources.

    Parameters
    ----------
    exp : `lsst.afw.image.Exposure`
        The exposure to characterize.
    snr : `float`
        The SNR threshold for detection.
    minPix : `int`
        The minimum number of pixels to count as a source.

    Returns
    -------
    result : `lsst.pipe.base.Struct`
        The result from the image characterization task.
    """
    charConfig = CharacterizeImageConfig()
    charConfig.doMeasurePsf = False
    charConfig.doApCorr = False
    charConfig.doDeblend = False
    charConfig.repair.doCosmicRay = False

    charConfig.detection.minPixels = minPix
    charConfig.detection.thresholdValue = snr
    charConfig.detection.includeThresholdMultiplier = 1
    charConfig.detection.nSigmaToGrow = 0

    charConfig.psfIterations = 1
    charConfig.installSimplePsf.fwhm = 5
    charConfig.installSimplePsf.width = 51
    
    # fit background with the most simple thing possible as we don't need
    # much sophistication here. weighting=False is *required* for very
    # large binSizes.
#     charConfig.background.algorithm = 'AKIMA'
    charConfig.background.approxOrderX = 1
    charConfig.background.approxOrderY = -1
    charConfig.background.binSize = max(exp.getWidth(), exp.getHeight())
    charConfig.background.weighting = False

    # set this to use all the same minimal settings as those above
    charConfig.detection.background = charConfig.background

    charTask = CharacterizeImageTask(config=charConfig)

    charResult = charTask.run(exp)
    return charResult

def solveFastCamera(filename, doPlot=False):
    exp = starTrackerFileToExposure(filename)
    exp.image.array = median_filter(exp.image.array, 2, mode='reflect')
    result = runCharactierizeImage(exp, 5, 25)
    sources = filterSourceCatOnBrightest(result.sourceCat, .5, minSources=5)
    if doPlot:
        plot(exp, sources, doSmooth=False)
    result = solver.run(exp, sources, True, useGaia=True, percentageScaleError=10, radius=5)
    if result is not None:
        fastWcs = result.wcs
        fastVisitInfo = exp.visitInfo
    else:
        print('fit failed')
        return None
    return fastWcs

In [ ]:
#Enter which image you want to look at
[camera, num] = ['Fast', 103]
date = 20230321
year = int(date/10000)
month = int((date - 10000 * year)/100)
day = int((date - 10000 * year - 100 * month))

path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
print(path)
filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
fast_exp = starTrackerFileToExposure(filename)
result = solveFastCamera(filename, doPlot=False)
fast_wcs = result

# This is a back-up
#from lsst.afw.geom import SkyWcs
#fast_wcs = SkyWcs.readFits('/home/mfisherlevine/temp/fastCamFits/GC103_O_20230321_001160_wcs.fits')

## Now plot and overlay the results to check the offsets

In [ ]:
# Now plot the data with matplotlib
%matplotlib inline
from matplotlib.path import Path
from matplotlib.patches import PathPatch
def fast_corners(fast_wcs, narrow_wcs):
    # This calculates where th fast camera is within the narrow camera image
    scale_ratio = fast_wcs.getPixelScale().asArcseconds() / narrow_wcs.getPixelScale().asArcseconds()
    theta = fast_wcs.getRelativeRotationToWcs(narrow_wcs).asRadians()
    center = narrow_wcs.skyToPixel(fast_wcs.getSkyOrigin())
    dx0 = fast_wcs.getPixelOrigin().x * scale_ratio
    dy0 = fast_wcs.getPixelOrigin().y * scale_ratio
    rot = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    corners = []
    for [i, j] in[[-1,-1],[-1,1],[1,1],[1,-1],[-1,-1]]:
        d0 = np.array([i*dx0, j*dy0])
        d = np.dot(rot, d0)
        dx = d[0]; dy = d[1]
        corners.append([center.x+dx, center.y+dy])   
    return corners

def alt_az_arrows(narrow_wcs, narrow_exp):
    # This calculates the Alt Az directions
    mData = narrow_exp.getInfo().getMetadata()
    time = Time((Time(mData["DATE-BEG"], scale='tai').unix_tai + Time(mData["DATE-END"], scale='tai').unix_tai)/2.0, \
                format='unix_tai', scale='tai')
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
    center = np.array((int(narrow_exp.image.array.shape[1] / 2), int(narrow_exp.image.array.shape[0] / 2)))
    center_raDec = narrow_wcs.pixelToSky(center[0], center[1])
    skyLocation = SkyCoord(center_raDec.getRa().asDegrees() * u.deg, center_raDec.getDec().asDegrees() * u.deg)
    obsAltAz = skyLocation.transform_to(altAz)
    az = obsAltAz.az.deg
    alt = obsAltAz.alt.deg
    azPlus = az + 500.0 / 3600.0 / np.cos(obsAltAz.alt.rad) # add 500 arcseconds
    altPlus = alt + 500.0 / 3600.0 # add 500 arcseconds
    altAzPlus = SkyCoord(AltAz(alt=alt*u.deg, az=azPlus*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDecAzPlus = altAzPlus.transform_to(ICRS)
    SpherePointAzPlus = SpherePoint(Angle(RaDecAzPlus.ra.rad), Angle(RaDecAzPlus.dec.rad))
    azPlusPixels = narrow_wcs.skyToPixel(SpherePointAzPlus)
    altPlusAz = SkyCoord(AltAz(alt=altPlus*u.deg, az=az*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDecAltPlus = altPlusAz.transform_to(ICRS)
    SpherePointAltPlus = SpherePoint(Angle(RaDecAltPlus.ra.rad), Angle(RaDecAltPlus.dec.rad))
    altPlusPixels = narrow_wcs.skyToPixel(SpherePointAltPlus)
    azPlusArrowLength = np.array(azPlusPixels) - np.array(center)
    altPlusArrowLength = np.array(altPlusPixels) - np.array(center)
    print(az, alt)
    print(mData["AZSTART"], mData["ELSTART"])
    print(center, azPlusPixels, altPlusPixels)
    return [center, azPlusArrowLength, altPlusArrowLength]

def offset_arrow(narrow_wcs, narrow_exp, deltaAlt, deltaAz):
    # This calculates the relative offset as a check
    mData = narrow_exp.getInfo().getMetadata()
    time = Time((Time(mData["DATE-BEG"], scale='tai').unix_tai + Time(mData["DATE-END"], scale='tai').unix_tai)/2.0, \
                format='unix_tai', scale='tai')
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
    #boresight = narrow_wcs.getPixelOrigin()
    boresight_raDec = narrow_wcs.pixelToSky(boresight.x, boresight.y)
    boresight_raDec = narrow_wcs.getSkyOrigin()
    skyLocation = SkyCoord(boresight_raDec.getRa().asDegrees() * u.deg, boresight_raDec.getDec().asDegrees() * u.deg)
    obsAltAz = skyLocation.transform_to(altAz)
    az = obsAltAz.az.deg
    alt = obsAltAz.alt.deg
    azPlus = az + deltaAz / 3600.0 / np.cos(obsAltAz.alt.rad)
    altPlus = alt + deltaAlt / 3600.0
    altAzPlus = SkyCoord(AltAz(alt=altPlus*u.deg, az=azPlus*u.deg, obstime=time, location=SIMONYI_LOCATION))
    RaDecAltAzPlus = altAzPlus.transform_to(ICRS)
    SpherePointAltAzPlus = SpherePoint(Angle(RaDecAltAzPlus.ra.rad), Angle(RaDecAltAzPlus.dec.rad))
    altAzPlusPixels = narrow_wcs.skyToPixel(SpherePointAltAzPlus)
    altAzPlusArrowLength = np.array(altAzPlusPixels) - np.array(boresight)
    return [boresight, altAzPlusArrowLength]


fig, ax = plt.subplots(figsize=(16,16))
ax.set_title(f"StarTracker Narrow and Fast, SeqNum {seqNum}")
ax.imshow(narrow_exp.image.array,  interpolation='Nearest', cmap='gray', vmin=0.1, vmax=1, origin='lower')
narrow_boresight = narrow_wcs.getPixelOrigin()
ax.scatter(narrow_boresight.x, narrow_boresight.y, \
            facecolors='cyan', edgecolors='cyan', marker='X', s=200, lw=1)
ax.text(narrow_boresight.x+50, narrow_boresight.y-10, "Boresight", color='cyan', fontsize=18)
[center, azPlusArrowLength, altPlusArrowLength] = alt_az_arrows(narrow_wcs, narrow_exp)
ax.arrow(center[0], center[1], azPlusArrowLength[0], azPlusArrowLength[1], color='lightgreen', \
        head_width=50, head_length=100, lw = 2)
ax.text(center[0] + azPlusArrowLength[0]+50, center[1] + azPlusArrowLength[1]-100, \
        'Plus AzCosAlt\n500"', color='lightgreen', fontsize=18)
ax.arrow(center[0], center[1], altPlusArrowLength[0], altPlusArrowLength[1], color='lightgreen', \
        head_width=50, head_length=100, lw = 2)
ax.text(center[0] + altPlusArrowLength[0]+100, center[1] + altPlusArrowLength[1], \
        'Plus Alt\n500"', color='lightgreen', fontsize=18)


path = Path(fast_corners(fast_wcs, narrow_wcs))
patch = PathPatch(path, facecolor='none', edgecolor='lightblue')
ax.add_patch(patch)
im = plt.imshow(fast_exp.image.array,  interpolation='Nearest', cmap='gray', vmin=1, vmax=1000, origin='lower', \
               extent=[0,narrow_exp.image.array.shape[1],0,narrow_exp.image.array.shape[0]], clip_path=patch, clip_on=True, alpha=0.5)
fast_center = narrow_wcs.skyToPixel(fast_wcs.getSkyOrigin())
ax.scatter(fast_center.x, fast_center.y, \
                    facecolors='lightblue', edgecolors='lightblue', marker='X', s=200, lw=1)
ax.text(fast_center.x+150, fast_center.y, "Fast Camera", color='lightblue', fontsize=18)
im.set_clip_path(patch)

[boresight, altAzPlusArrowLength] = offset_arrow(narrow_wcs, narrow_exp, median_offset[0], median_offset[1])
ax.arrow(boresight.x, boresight.y, altAzPlusArrowLength[0], altAzPlusArrowLength[1], color='pink', \
        head_width=10, head_length=20, lw = 2)
ax.text(boresight.x-700, boresight.y-400, \
        f"Median offset\nAlt = {median_offset[0]:.1f}\nAzCosAlt = {median_offset[1]:.1f}\nPixelX = {fast_center.x-boresight.x:.0f}\nPixelY = {fast_center.y-boresight.y:.0f}", \
        color='pink', fontsize=18)

print(f"Pixel_offsets = ({fast_center.x-boresight.x}, {fast_center.y-boresight.y})")

plt.savefig(f"/home/craiglagegit/DATA/StarTracker_Fast_Offset_{date}_{seqNum}.pdf")